In [1]:
import hopsworks
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/Users/hugolindgren/miniconda3/envs/scalable_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
#ML imports
#import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

OSError: dlopen(/Users/hugolindgren/miniconda3/envs/scalable_project/lib/python3.10/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <3AF1EF0C-311C-31EC-BCE3-679F37ABEE16> /Users/hugolindgren/miniconda3/envs/scalable_project/lib/python3.10/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [9]:
import datetime
year = datetime.date.today().year
year

2023

In [10]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/224406
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view

valuation_fg = fs.get_feature_group(name="valuationdataset", version=year-2022)
query = valuation_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="valuationdataset_fv",
                                  version=year-2022,
                                  description="Read from valuation dataset",
                                  labels=["market_value_in_eur"],
                                  query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/224406/fs/224325/fv/valuationdataset_fv/version/1


In [22]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)



Finished: Reading data from Hopsworks, using ArrowFlight (1.20s) 


In [26]:
#Drop date column, keeping it because might make sense later

X_train = X_train.drop(columns=["player_id", "date"])
X_test = X_test.drop(columns=["player_id", "date"])

In [27]:
X_train

,yellow_cards,red_cards,goals,assists,minutes_played,height_in_cm,age,months_left,own_goals,opponent_goals,clean_sheets,league_es1,league_fr1,league_gb1,league_it1,league_l1,position_attack,position_defender,position_goalkeeper,position_midfield
0,1,0,0,2,682,170.0,22.0,37,98,66,15,False,False,True,False,False,False,False,False,True
1,2,0,0,0,90,177.0,20.0,25,52,56,10,False,False,False,True,False,False,False,False,True
2,0,0,0,0,46,186.0,26.0,13,47,61,9,False,False,False,True,False,False,True,False,False
3,6,0,1,9,3534,173.0,33.0,25,75,38,15,False,False,True,False,False,False,True,False,False
4,1,0,0,0,130,187.0,19.0,49,53,57,11,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,2,0,0,0,948,176.0,29.0,37,83,47,14,True,False,False,False,False,False,False,False,True
2710,0,0,0,0,21,177.0,33.0,25,45,44,10,False,True,False,False,False,False,False,False,True
2711,4,0,7,8,2914,183.0,31.0,49,70,42,23,False,False,False,True,False,False,False,False,True
2712,0,0,0,0,44,176.0,20.0,13,44,41,14,False,True,False,False,False,False,True,False,False


In [33]:
# Create the XGBoost data containers
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define the parameters for the model
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.1,  # the training step for each iteration
    'objective': 'reg:squarederror',  # regression with squared loss
}
num_round = 100  # the number of training iterations

# Train the model
bst = xgb.train(param, dtrain, num_round)

# Predict on the test set
y_pred = bst.predict(dtest)

# Evaluate the model using RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse.round()}')




RMSE: 9486181.0
